<a href="https://colab.research.google.com/github/tremblay17/Machine-Learning-Projects/blob/main/StockClassifier-Optuna-Keras-v1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install yfinance
%pip install pandas_ta
%pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 527 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 21.8 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=9d8ca5c74215c132044a25db98c2dc338a0693551b08607d90dc687b02e9301e
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 20.1 MB/s 
     |████████████████████████████████| 209 kB 56.4 MB/s 
     |████████████████████████████████| 81 kB 11.8 MB/s 
     |████████████████████████████████| 78 kB 9.8 MB/s 
     |████████████████████████████████| 147 kB 62.6 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 112 kB 60.0 MB/s 
  Created wheel for pyperclip: filename=pypercl

In [ ]:
from sklearn.model_selection import train_test_split
import keras as k
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.metrics import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris, load_digits
import yfinance as yf
import pandas_ta as ta
import optuna as o

In [ ]:
from tensorflow.python.framework.ops import default_session
np.set_printoptions(precision=3, suppress=True)

startDate = "2022-12-06" #@param {type:"date"}
endDate = "2022-12-06" #@param {type: "date"}
stock = 'NVDA' #@param {type: "string"}
period = '10y' #@param ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y']
interval= '1d' #@param ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']
toDate_historical = 'toDate' #@param ['toDate', 'historical']
length_days = 14 #@param {type:"slider", min:1, max:200, step:1}
macdFast = 12 #@param {type:"slider", min:1, max:200, step:1}
macdSlow = 26 #@param {type:"slider", min:1, max:200, step:1}
macdSignal = 9 #@param {type:"slider", min:1, max:200, step:1}
stochFastK = 14 #@param {type:"slider", min:1, max:200, step:1}
stochSlowK = 3 #@param {type:"slider", min:1, max:200, step:1}
stochD = 3 #@param {type:"slider", min:1, max:200, step:1}

def loadData():
  if(toDate_historical == 'toDate'):
    data = yf.Ticker(stock).history(period=period, interval=interval)[
        ['Close', 'Open', 'High', 'Volume', 'Low']]
  else:
    data = yf.Ticker(stock).history(start=startDate, end=endDate, interval=interval)[
        ['Close', 'Open', 'High', 'Volume', 'Low']]

  data.ta.macd(close='close', fast=macdFast, slow=macdSlow, signal=macdSignal, append=True)
  data.ta.adx(length=length_days, append=True)
  data.ta.rsi(length=length_days, append=True)
  data.ta.bbands(length=length_days, append=True)
  data.ta.obv(append=True)
  data.ta.stoch(high='High', low='Low', fast_k=stochFastK, slow_k=stochSlowK, slow_d=stochD, append=True)
  data.ta.aroon(length=length_days, append=True)

  print(data.info())

  Class_Cat = []
  Class_Int = []
  for macd, macdh, macds, adx, dmp, dmn, rsi, bbl, bbm, bbu, bbb, bbp, obv, stochk, stochd, aroond, aroonu, aroonosc in data.iloc[:, 5:].values:
    if ((macd == 'NaN') or (macdh == 'NaN') or (macds == 'NaN') or (adx == 'NaN') or (dmp == 'NaN') or (dmn == 'NaN') or (rsi == 'NaN')):
      Class_Cat.append('Hold')
      Class_Int.append(1)
    elif (((macd >= macds) and (macdh > 0) and ((adx >= 25) and (dmp > dmn))) or (rsi <= 30)):
      Class_Cat.append('Buy')
      Class_Int.append(0)
    elif (((macd <= macds) and (macdh < 0) and ((adx >= 25) and (dmn > dmp))) or (rsi >= 70)):
      Class_Cat.append('Sell')
      Class_Int.append(2)
    else:
      Class_Cat.append('Hold')
      Class_Int.append(1)

  data['Class_Int'] = Class_Int
  data['Class_Cat'] = Class_Cat
  data = data.iloc[34:, :]
  return data

data = loadData()
data.head(60)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2517 entries, 2012-12-07 00:00:00-05:00 to 2022-12-06 00:00:00-05:00
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Close          2517 non-null   float64
 1   Open           2517 non-null   float64
 2   High           2517 non-null   float64
 3   Volume         2517 non-null   int64  
 4   Low            2517 non-null   float64
 5   MACD_12_26_9   2492 non-null   float64
 6   MACDh_12_26_9  2484 non-null   float64
 7   MACDs_12_26_9  2484 non-null   float64
 8   ADX_14         2490 non-null   float64
 9   DMP_14         2503 non-null   float64
 10  DMN_14         2503 non-null   float64
 11  RSI_14         2503 non-null   float64
 12  BBL_14_2.0     2504 non-null   float64
 13  BBM_14_2.0     2504 non-null   float64
 14  BBU_14_2.0     2504 non-null   float64
 15  BBB_14_2.0     2504 non-null   float64
 16  BBP_14_2.0     2504 non-null   float64
 17  OBV 

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,...,BBB_14_2.0,BBP_14_2.0,OBV,STOCHk_14_3_3,STOCHd_14_3_3,AROOND_14,AROONU_14,AROONOSC_14,Class_Int,Class_Cat
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-29 00:00:00-05:00,2.849532,2.900334,2.907261,38887200,2.837986,-0.009388,0.010009,-0.019397,12.500610,25.873466,...,4.807163,0.698138,140192000.0,56.589050,42.745965,35.714286,0.000000,-35.714286,1,Hold
2013-01-30 00:00:00-05:00,2.844914,2.851842,2.891098,28759200,2.835677,-0.008529,0.008694,-0.017223,12.603817,24.141832,...,4.869230,0.648720,111432800.0,60.987190,54.360292,28.571429,85.714286,57.142857,1,Hold
2013-01-31 00:00:00-05:00,2.831059,2.844914,2.877243,47054400,2.812586,-0.008865,0.006687,-0.015552,12.041033,22.269265,...,4.874257,0.544328,64378400.0,50.855040,56.143760,21.428571,78.571429,57.142857,1,Hold
2013-02-01 00:00:00-05:00,2.856459,2.854150,2.865696,28881600,2.831058,-0.007000,0.006841,-0.013841,11.535413,21.315420,...,4.995099,0.704364,93260000.0,52.137124,54.659785,14.285714,71.428571,57.142857,1,Hold
2013-02-04 00:00:00-05:00,2.807966,2.840295,2.898025,51174400,2.805657,-0.009328,0.003611,-0.012939,11.849593,22.814330,...,5.034926,0.366733,42085600.0,45.299374,49.430513,7.142857,64.285714,57.142857,1,Hold
2013-02-05 00:00:00-05:00,2.872623,2.821822,2.888788,36729600,2.812585,-0.005888,0.005641,-0.011529,12.133388,20.679884,...,4.671775,0.787724,78815200.0,52.259684,49.898727,0.000000,57.142857,57.142857,1,Hold
2013-02-06 00:00:00-05:00,2.849532,2.803348,2.872623,53068800,2.787184,-0.004968,0.005249,-0.010216,11.618553,18.689264,...,4.376707,0.587620,25746400.0,49.026617,48.861892,0.000000,50.000000,50.000000,1,Hold
2013-02-07 00:00:00-05:00,2.837986,2.840295,2.863387,44000000,2.810276,-0.005111,0.004084,-0.009195,11.151459,17.557850,...,4.358876,0.489338,-18253600.0,52.352507,51.212936,21.428571,42.857143,21.428571,1,Hold
2013-02-08 00:00:00-05:00,2.856459,2.856459,2.879551,30723200,2.837986,-0.003691,0.004403,-0.008094,11.156448,18.593381,...,4.238459,0.614996,12469600.0,48.309095,49.896073,14.285714,35.714286,21.428571,1,Hold


In [ ]:
data_inputs = data.iloc[:, 0:23].values
data_class = data.iloc[:, 23:24].values
print(data_class)


[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [1]]


In [ ]:
# Classes 'Buy(1)', 'Hold(0)', or 'Sell(2)'
np.asarray(data_inputs).astype('float64')
np.asarray(data_class).astype('float64')


array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]])

In [ ]:

sc = MinMaxScaler(feature_range=(0, 1))


(1862, 23)

(1862, 23, 1)

In [ ]:
from keras.losses import *

def objective(trial):
  x_train, x_test, y_train, y_test = train_test_split(
    data_inputs, data_class, train_size=0.75, shuffle=True, random_state=40)
  sc.fit(x_train)
  sc.transform(x_train)
  sc.transform(x_test)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  batchSize = trial.suggest_int('batchSize', 25, 150)  ##param {type:"slider", min:1, max:200, step:1}
  epochs = trial.suggest_int('epochs', 10, 200)  ##param {type:"slider", min:1, max:200, step:1}

  #Currently generates the highest accuracy
  #{'layer1': 75, 'activation1': 'tanh', 'layer2': 85, 'activation2': 'softsign', 'layer3': 73, 'activation3': 'softsign', 'layer4': 29, 'activation4': 'relu', 'lr': 0.006548463552532551}
  seqModel = Sequential(
      [
          LSTM(units=trial.suggest_int('layer1', 24, 85),
               activation=trial.suggest_categorical('activation1', ['tanh','selu','relu','softsign','softmax']),
               return_sequences=True,
               input_shape=(x_train.shape[1], 1)),

          LSTM(units=trial.suggest_int('layer2', 24, 85),
               activation=trial.suggest_categorical('activation2', ['tanh','selu','relu','softsign','softmax']),
               return_sequences=True),

          LSTM(units=trial.suggest_int('layer3', 24, 85),
               activation=trial.suggest_categorical('activation3', ['tanh','selu','relu','softsign','softmax']),
               return_sequences=True),

          LSTM(units=trial.suggest_int('layer4', 24, 85),
               activation=trial.suggest_categorical('activation4', ['tanh','selu','relu','softsign','softmax'])),

          Dense(units=3, activation='sigmoid')
      ]
  )

  learningRate = trial.suggest_float('lr', 0.0001, 0.1, log=True)

  optimizer = Adam(learning_rate=learningRate) #@param ["Adam(learning_rate=learningRate)", "Adadelta(learning_rate=learningRate)", "Adamax(learning_rate=learningRate)", "Nadam(learning_rate=learningRate)", 'SGD(learning_rate=learningRate)'] {type:"raw"}
  lossFunc = trial.suggest_categorical('lossFunc', [SparseCategoricalCrossentropy(), Poisson(), KLDivergence()])
  metrics = [SparseCategoricalAccuracy(), MeanSquaredError()]
  seqModel.compile(optimizer=optimizer, loss=lossFunc, metrics=metrics)

  seqModel.fit(x_train,
             y_train,
             validation_data=(x_test, y_test),
             epochs=epochs,
             batch_size=batchSize,
             validation_batch_size=batchSize,
             verbose=True)

  score = seqModel.evaluate(x_test, y_test, verbose=0)
  print(seqModel.metrics_names[0], score[0], seqModel.metrics_names[1], score[1], seqModel.metrics_names[2], score[2])

  return score[1]

study = o.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-07 06:18:32,268] A new study created in memory with name: no-name-05e10ae6-2930-4296-9b7f-5a8e43306953


Epoch 1/12
75/75 [==============================] - 12s 73ms/step - loss: 0.8628 - sparse_categorical_accuracy: 0.6702 - mean_squared_error: 0.5512 - val_loss: 0.8577 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5768
Epoch 2/12
75/75 [==============================] - 4s 57ms/step - loss: 0.8311 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5643 - val_loss: 0.8097 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.6169
Epoch 3/12
75/75 [==============================] - 4s 58ms/step - loss: 0.7854 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5837 - val_loss: 0.7667 - val_sparse_categorical_accuracy: 0.6779 - val_mean_squared_error: 0.6615
Epoch 4/12
75/75 [==============================] - 3s 43ms/step - loss: 0.6802 - sparse_categorical_accuracy: 0.7100 - mean_squared_error: 0.6197 - val_loss: 0.6597 - val_sparse_categorical_accuracy: 0.7118 - val_mean_squared_error: 0.6865
Epoch 5/12
75/75 [=================

[I 2022-12-07 06:19:23,464] Trial 0 finished with value: 0.7697262763977051 and parameters: {'layer1': 56, 'activation1': 'tanh', 'layer2': 64, 'activation2': 'selu', 'layer3': 24, 'activation3': 'tanh', 'layer4': 37, 'activation4': 'relu', 'lr': 0.0005352730222370649}. Best is trial 0 with value: 0.7697262763977051.


loss 0.5556195378303528 sparse_categorical_accuracy 0.7697262763977051 mean_squared_error 0.6976708173751831
Epoch 1/12
75/75 [==============================] - 9s 56ms/step - loss: 1.0976 - sparse_categorical_accuracy: 0.4039 - mean_squared_error: 0.4911 - val_loss: 1.0764 - val_sparse_categorical_accuracy: 0.6731 - val_mean_squared_error: 0.5199
Epoch 2/12
75/75 [==============================] - 3s 46ms/step - loss: 1.0559 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.4907 - val_loss: 1.0350 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5189
Epoch 3/12
75/75 [==============================] - 3s 43ms/step - loss: 1.0084 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.4889 - val_loss: 0.9815 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5183
Epoch 4/12
75/75 [==============================] - 3s 45ms/step - loss: 0.9476 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.4902 - val_loss: 0.9193 - 

[I 2022-12-07 06:20:09,360] Trial 1 finished with value: 0.6682769656181335 and parameters: {'layer1': 51, 'activation1': 'softsign', 'layer2': 27, 'activation2': 'selu', 'layer3': 41, 'activation3': 'relu', 'layer4': 62, 'activation4': 'tanh', 'lr': 1.230772735069921e-05}. Best is trial 0 with value: 0.7697262763977051.


...

loss 0.40671080350875854 sparse_categorical_accuracy 0.8341385126113892 mean_squared_error 0.6972536444664001
Epoch 1/12
75/75 [==============================] - 10s 69ms/step - loss: 0.8871 - sparse_categorical_accuracy: 0.6697 - mean_squared_error: 0.6400 - val_loss: 0.8711 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.6519
Epoch 2/12
75/75 [==============================] - 4s 57ms/step - loss: 0.8546 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.6318 - val_loss: 0.8711 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.6723
Epoch 3/12
75/75 [==============================] - 4s 56ms/step - loss: 0.8548 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.6233 - val_loss: 0.8633 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.6418
Epoch 4/12
75/75 [==============================] - 4s 57ms/step - loss: 0.8503 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.6146 - val_loss: 0.8616 

[I 2022-12-07 07:35:44,305] Trial 76 finished with value: 0.6682769656181335 and parameters: {'layer1': 66, 'activation1': 'tanh', 'layer2': 83, 'activation2': 'relu', 'layer3': 78, 'activation3': 'softsign', 'layer4': 33, 'activation4': 'relu', 'lr': 0.013094318466157046}. Best is trial 72 with value: 0.8534621596336365.


loss 0.8608148694038391 sparse_categorical_accuracy 0.6682769656181335 mean_squared_error 0.5958863496780396
Epoch 1/12
75/75 [==============================] - 11s 78ms/step - loss: 0.8910 - sparse_categorical_accuracy: 0.6660 - mean_squared_error: 0.5401 - val_loss: 0.8962 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5376
Epoch 2/12
75/75 [==============================] - 5s 70ms/step - loss: 0.8531 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5361 - val_loss: 0.8743 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5512
Epoch 3/12
75/75 [==============================] - 5s 71ms/step - loss: 0.8538 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5365 - val_loss: 0.8630 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5784
Epoch 4/12
75/75 [==============================] - 5s 68ms/step - loss: 0.8515 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5440 - val_loss: 0.8610 -

[I 2022-12-07 07:36:53,298] Trial 77 finished with value: 0.6682769656181335 and parameters: {'layer1': 77, 'activation1': 'relu', 'layer2': 97, 'activation2': 'softsign', 'layer3': 63, 'activation3': 'selu', 'layer4': 89, 'activation4': 'relu', 'lr': 0.011701625371049205}. Best is trial 72 with value: 0.8534621596336365.


loss 0.8638953566551208 sparse_categorical_accuracy 0.6682769656181335 mean_squared_error 0.5681334137916565
Epoch 1/12
75/75 [==============================] - 11s 76ms/step - loss: 0.8822 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5127 - val_loss: 0.8625 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5586
Epoch 2/12
75/75 [==============================] - 5s 63ms/step - loss: 0.8486 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5276 - val_loss: 0.8612 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5525
Epoch 3/12
75/75 [==============================] - 5s 63ms/step - loss: 0.8482 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5252 - val_loss: 0.8625 - val_sparse_categorical_accuracy: 0.6683 - val_mean_squared_error: 0.5555
Epoch 4/12
75/75 [==============================] - 5s 63ms/step - loss: 0.8482 - sparse_categorical_accuracy: 0.6740 - mean_squared_error: 0.5272 - val_loss: 0.8612 -

NameError: ignored

In [ ]:
minMSE = trial

In [ ]:
minLoss = trial

In [ ]:
print(seqModel.evaluate(x_train, y_train))
seqModel.evaluate(x_test, y_test)

59/59 [==============================] - 1s 11ms/step - loss: 0.4904 - sparse_categorical_accuracy: 0.8077 - mean_squared_error: 0.6174
[0.49036017060279846, 0.8077335953712463, 0.6174330115318298]
20/20 [==============================] - 0s 11ms/step - loss: 0.5210 - sparse_categorical_accuracy: 0.7971 - mean_squared_error: 0.6003


[0.521033763885498, 0.7971014380455017, 0.6002863645553589]

In [ ]:
print(seqModel.predict(x_test))
seqModel.predict(x_train)

20/20 [==============================] - 2s 11ms/step
[[0.179 0.84  0.461]
 [0.733 0.407 0.478]
 [0.064 0.982 0.27 ]
 ...
 [0.286 0.835 0.47 ]
 [0.448 0.896 0.405]
 [0.139 0.933 0.425]]
59/59 [==============================] - 1s 13ms/step


array([[0.068, 0.984, 0.263],
       [0.065, 0.987, 0.246],
       [0.748, 0.437, 0.437],
       ...,
       [0.52 , 0.557, 0.493],
       [0.07 , 0.98 , 0.278],
       [0.348, 0.699, 0.47 ]], dtype=float32)

In [ ]:
np.set_printoptions(precision=3, suppress=True)

data = yf.Ticker(stock).history(period=period)[
    ['Close', 'Open', 'High', 'Volume', 'Low']]
data.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)
data.ta.adx(length=14, append=True)
data.ta.rsi(length=14, append=True)

Class_Cat = []
Class_Int = []

for macd, macdh, macds, adx, dmp, dmn, rsi in data.iloc[:, 5:].values:
  if ((macd == 'NaN') or (macdh == 'NaN') or (macds == 'NaN') or (adx == 'NaN') or (dmp == 'NaN') or (dmn == 'NaN') or (rsi == 'NaN')):
    Class_Cat.append('Hold')
    Class_Int.append(1)
  elif (((macd >= macds) and (macdh > 0) and ((adx >= 25) and (dmp > dmn))) or (rsi <= 30)):
    Class_Cat.append('Buy')
    Class_Int.append(0)
  elif (((macd <= macds) and (macdh < 0) and ((adx >= 25) and (dmn > dmp))) or (rsi >= 70)):
    Class_Cat.append('Sell')
    Class_Int.append(2)
  else:
    Class_Cat.append('Hold')
    Class_Int.append(1)

data['Class_Int'] = Class_Int
data['Class_Cat'] = Class_Cat
data = data.iloc[34:, :]

data_inputs = data.iloc[:, 0:12].values
data_class = data.iloc[:, 12:13].values
print(data_class)
# Classes 'Buy(1)', 'Hold(0)', or 'Sell(2)'
np.asarray(data_inputs).astype('float64')
np.asarray(data_class).astype('float64')
sc = MinMaxScaler(feature_range=(0, 1))
x_train, x_test, y_train, y_test = train_test_split(
    data_inputs, data_class, train_size=0.5)
# Normalization
sc.fit(x_train)
sc.transform(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

data.head(5)


[[1]
 [2]
 [2]
 ...
 [1]
 [1]
 [1]]


,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,DMN_14,RSI_14,Class_Int,Class_Cat
Date,,,,,,,,,,,,,,
2018-01-29 00:00:00-05:00,39.928284,40.451278,40.451278,202561600,39.716709,-0.015054,-0.276953,0.261900,24.745524,21.258921,35.916006,33.391485,1,Hold
2018-01-30 00:00:00-05:00,39.692932,39.350608,39.788021,184192800,39.153296,-0.141343,-0.322594,0.181251,25.590727,19.270459,39.356573,31.591500,2,Sell
2018-01-31 00:00:00-05:00,39.802288,39.669163,40.042393,129915600,39.581206,-0.229954,-0.328964,0.099010,25.721926,21.299910,37.130818,33.388275,2,Sell
2018-02-01 00:00:00-05:00,39.885487,39.740474,40.085175,188923200,39.643006,-0.290121,-0.311305,0.021184,25.735198,20.658584,35.082283,34.791559,2,Sell
2018-02-02 00:00:00-05:00,38.154850,39.462337,39.652518,346375200,38.059761,-0.472010,-0.394555,-0.077455,27.481689,16.588915,45.254472,23.636832,0,Buy


In [ ]:
print(seqModel.predict(x_test))


20/20 [==============================] - 0s 7ms/step
[[0.358 0.744 0.437]
 [0.069 0.918 0.545]
 [0.087 0.933 0.449]
 ...
 [0.097 0.879 0.604]
 [0.205 0.813 0.525]
 [0.06  0.935 0.509]]
